In [1]:
import pandas as pd
df = pd.read_csv('cleaned_responses.csv')
print(df.columns)

Index(['Unnamed: 0', 'first_name', 'last_name', 'Email', 'class_year',
       'genre_1', 'genre_1_other', 'genre_2', 'genre_2_other', 'genre_3',
       'genre_3_other', 'genre_4', 'genre_4_other', 'genre_5', 'genre_5_other',
       'num_genres', 'songs_1', 'songs_2', 'songs_3', 'songs_4', 'songs_5',
       'artist_1', 'artist_1_other', 'artist_1_min', 'artist_2',
       'artist_2_other', 'artist_3', 'artist_3_other', 'artist_4',
       'artist_4_other', 'artist_5', 'artist_5_other', 'min_total',
       'top_song_plays', 'aura_1', 'aura_2'],
      dtype='object')


In [2]:
import numpy as np
from scipy.sparse import csr_matrix

In [3]:
genre_counts = {}
genre_cols = ['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',]
for col in genre_cols:
    for row in df[col]:
        genre_counts[row] = genre_counts.get(row, 0) + 1

#print(genre_counts)

print({k: v for k, v in sorted(genre_counts.items(), key=lambda item: item[1])})

lone_genres = set()
for key, value in genre_counts.items():
    if value == 1:
        lone_genres.add(key)
        
print(str(len(lone_genres)) + " lone genres out of " + str(len(genre_counts)))


{'singer-songwriter': 1, 'traprun': 1, 'pop latino': 1, 'new wave ': 1, 'detroit rap': 1, 'rabindra sangeet': 1, 'brazilian popular music (mpb)': 1, 'kanye west': 1, 'bubblegrunge': 1, 'post rock': 1, 'indiepop': 1, 'bass house': 1, 'indie': 1, 'kpop': 1, 'hip-hop/rap': 1, 'doom metal': 1, 'pop dance': 1, 'uk hip hop': 1, 'corrido': 1, 'deep house': 1, 'undergroup hip hop': 1, 'eurovision': 1, 'korean indie rock': 1, 'uk alternative pop': 1, 'anti foll': 1, 'baroque ': 1, 'glee club': 1, 'dance pop ': 1, 'pop rwp': 1, 'the weeknd': 1, 'swing': 1, 'trova': 1, 'pop-rap': 1, 'post punk': 1, 'grime': 1, 'death metal': 1, 'r&b/soul': 1, "video game music [this shouldn't have been this high up :(( ]": 1, 'small room': 1, 'bossa nova': 1, 'compositional ambient': 1, 'sludge metal': 1, 'filmi': 1, 'banda': 1, 'jazz rap': 1, 'sad lo-fi': 1, 'rock en espanol': 1, 'pop edm': 1, 'vocaloid': 1, 'epicore': 1, 'dfw rap': 1, 'pop urbaine': 1, 'japanese math rock': 1, 'opera ': 1, 'contemporary r&b': 1

In [6]:
threshold = 5
less_than = 0 # holds number of people with all genres of less than the threshold amount
num_rows = 0 

for index, row in df.iterrows():
    num_rows += 1
    if sum([genre_counts[row[genre_col]] < threshold for genre_col in genre_cols]) == 5:
        less_than += 1
        print(row["Email"])
        

scherea@mit.edu
favoured@mit.edu
mimisuar@mit.edu
j_luera@mit.edu


In [23]:
genre_counts = {}
genre_cols = ['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',]

for _, row in df.iterrows():
    #print(row)
    genre_counts[(row[5], row[7])] = genre_counts.get((row[5], row[7]), 0) + 1
    genre_counts[(row[7], row[5])] = genre_counts.get((row[7], row[5]), 0) + 1
    
print({k: v for k, v in sorted(genre_counts.items(), key=lambda item: item[1], reverse=True)})


{('dance pop', 'indie pop'): 18, ('indie pop', 'dance pop'): 18, ('dance pop', 'k-pop'): 12, ('k-pop', 'dance pop'): 12, ('dance pop', 'pop rap'): 11, ('pop rap', 'dance pop'): 11, ('c-pop', 'k-pop'): 6, ('k-pop', 'c-pop'): 6, ('pop rap', 'melodic rap'): 5, ('melodic rap', 'pop rap'): 5, ('dance pop', 'show tunes'): 4, ('show tunes', 'dance pop'): 4, ('dance pop', 'alternative r&b'): 4, ('alternative r&b', 'dance pop'): 4, ('alternative r&b', 'indie pop'): 3, ('indie pop', 'alternative r&b'): 3, ('indie pop', 'electropop'): 3, ('electropop', 'indie pop'): 3, ('dance pop', 'indie rock'): 3, ('indie rock', 'dance pop'): 3, ('dance pop', 'reggaeton'): 3, ('reggaeton', 'dance pop'): 3, ('dance pop', 'chill r&b'): 3, ('chill r&b', 'dance pop'): 3, ('dance pop', 'electropop'): 3, ('electropop', 'dance pop'): 3, ('lo-fi beats', 'indie pop'): 3, ('indie pop', 'lo-fi beats'): 3, ('contemporary country', 'dance pop'): 3, ('dance pop', 'contemporary country'): 3, ('dance pop', 'modern bollywood')

In [3]:
spotify_to_ind, ind_to_spotify = {}, {}
spotify_genres = pd.unique(df[['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5']].values.ravel('K'))
spotify_artists = pd.unique(df[['artist_1', 'artist_2', 'artist_3', 'artist_4', 'artist_5']].values.ravel('K'))
spotify_all = list(spotify_genres) + list(spotify_artists)
for ind in range(len(spotify_all)):
    spotify_to_ind[spotify_all[ind]] = ind

In [11]:
X_dense = np.zeros((len(df), len(spotify_all)))
for index, row in df.iterrows():
    for col in ['genre', 'artist']:
        for i in range(5):
            if col == 'genre':
                X_dense[index, spotify_to_ind[row[col + '_' + str(5-i)]]] = 2**i
            else:
                X_dense[index, spotify_to_ind[row[col + '_' + str(5-i)]]] = i+1
X = csr_matrix(X_dense)    

In [12]:
#make genre artst matrix
#loop thru all the rows of df, set top 5 (16, 8, 4, 2, 1)
#reduce using SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
X_red = svd.fit_transform(X)

from k_means_constrained import KMeansConstrained
clf = KMeansConstrained(
     n_clusters=40,
     size_min=5,
     size_max=10,
     random_state=0
)
clf.fit_predict(X_red)
print(clf.cluster_centers_)
print(clf.labels_)


[[ 1.65236060e+00 -9.43099248e-02 -2.08567859e-01 ... -5.66100075e-02
  -3.23667897e-01 -5.21769981e-01]
 [ 4.68865339e+00  1.49827306e+01  1.10162471e+00 ... -1.27920716e-01
  -1.83810660e-01 -1.23836868e-01]
 [ 1.64956552e+01 -4.34058612e-01 -1.71676299e+00 ... -6.29625392e-02
  -3.76848075e-02 -3.17264159e-01]
 ...
 [ 1.11329088e+01  1.38141582e+01  1.41940405e-01 ...  5.94370280e-02
  -2.25605732e-01 -2.08746598e-01]
 [ 1.59439326e+01 -1.53609025e+00 -2.20669329e+00 ... -9.59646570e-03
  -2.22240239e-01  4.07331338e-01]
 [ 9.38988170e+00 -2.28006090e+00 -1.53279251e+00 ... -1.10980392e+00
  -5.83528343e-01  2.61228572e-02]]
[21  7  6  0 25 34 24  4 39 21 25 12 19  3 28 24 11 11  9  6  8 37 33 26
 26 13 20  2 10  4  9  8 17 31  1  6  0 10  2 36 11  2  4 26 33 39  9  2
 38 39 12 21 24 38  4 34 32  7  9  7  9 22 25 31 12  2  7  5 34 27 37 11
 10 34 11 36 10 35 19  5 35 19 14 38 21 21 17 37  6 27 34 13 30 27 33  8
  2  6 24  1 10 29  9 35 24 28 33 16  7 14 36 33 32 31  0 29 35  1 14 15

In [13]:
df['cluster'] = clf.labels_

In [14]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()
with pd.ExcelWriter('output.xlsx') as writer:
    for cluster in range(40):
        select_df = df[df['cluster']==cluster]
        select_df.to_excel(writer, sheet_name='Sheet'+str(cluster+1))
    writer.save()